<a href="https://colab.research.google.com/github/RobinSmits/FakeNews-Generator-And-Detector/blob/main/FakeNews_Generator_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we start our process of creating Fake News. In summary we do the following actions:

1.   Download a news dataset and split it in 2 parts.
2.   Train a T5 model to generate fake news based on the first dataset.
3.   Use the T5 model to generate fake news based on the second dataset.

By splitting the dataset we make sure that there is no 'leakage'. Generating the fake news will be done on data that the model hasn't seen during training.

The generated fake news will be used in the second notebook to train a RoBERTa classifier model to recognize real and fake news.

In the final notebook we use a test set to generate fake news with the T5 model and next we try to classify the real and fake news with the RoBERTa classifier.


** Note !! **

This notebook can only run on GPU/CPU. Where GPU is the preferred solution.
TPU would be possible but then you would need to modify the code of getting the news dataset manually and create a TF.data.Dataset.

When running the current code on TPU it will fail  with the error 'UnimplementedError: File system scheme '[local]' not implemented (file: '/root/tensorflow_datasets/ag_news_subset/1.0.0/ag_news_subset-train.tfrecord-00000-of-00001')'. 

Also generating text on the TPU is way to slow.
Overall using GPU to Train and to Generate text is the preferred way to use this notebook.
But offcourse feel free to experiment with that ;-)


In [ ]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm

# Install Specific Versions
!pip install tensorflow==2.4.1
!pip install tensorflow-datasets==4.1.0
!pip install transformers==4.2.2
!pip install sentencepiece==0.1.95

# Import Packages
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import *
import sentencepiece

     |████████████████████████████████| 3.6MB 7.9MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
     |████████████████████████████████| 1.8MB 8.6MB/s 
     |████████████████████████████████| 2.9MB 44.2MB/s 
     |████████████████████████████████| 890kB 65.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3fbea284e497994a0e01f73a38d9efef297fe4147a5364068065f485d203cab8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 7.3MB/s 


I've created and tested these notebooks on Google Colab Pro and used Google Drive to store and load any files created. 

If you run the code locally on a computer then modify the 'WORK_DIR' accordingly. Google Drive will not be needed in that case.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set Folder to use...
WORK_DIR = '/content/drive/My Drive/fake_news/'
os.makedirs(WORK_DIR, exist_ok = True) 

Mounted at /content/drive


Next we set some config for the device to use (GPU / CPU all work..). Also we set some constants. For the learning rate you could try different settings. If the model generates only garbage...then likely the learning rate was set to high.

You can set 2 actions:

1.   PERFORM_TRAINING: Set to True to Train a T5 Model from scratch.
2.   GENERATE_TEXT: Set to True to use a T5 Model to generate a text file with fake news. Note that generation can take a long time (multiple hours). Note that text generation on TPU is extremely slow...so pick a GPU todo the text generation on.

And finally we set the T5 tokenizer and config to use.

In [ ]:
# Configure Strategy. Assume TPU...if not set default for GPU/CPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy()

# Set Auto Tune
AUTO = tf.data.experimental.AUTOTUNE

# Supress Warnings
tf.autograph.set_verbosity(0, False)

# Constants
MAX_LEN = 512     # Max number of tokens for T5 to use.
EPOCHS = 1
VERBOSE = 1
TRAIN_SPLITS = 2

# Set Actions
PERFORM_TRAINING = True
GENERATE_TEXT = True

# Batch Size
GENERATE_BATCH_SIZE = 30 * strategy.num_replicas_in_sync
BATCH_SIZE = 4 * strategy.num_replicas_in_sync
print(f'Train Batch Size: {BATCH_SIZE}')
print(f'Generate Batch Size: {GENERATE_BATCH_SIZE}')

# Learning Rate
LR = 1e-4 * strategy.num_replicas_in_sync
print('Learning Rate: {}'.format(LR))

# Set T5 Type
t5_size = 't5-base'     
print(f'T5 Model Type: {t5_size}')

# Set T5 Task Name
task_name = 'generate fake news: '
print(f'T5 Task Name: {task_name}')

# Set T5 Config
t5_config = T5Config.from_pretrained(t5_size)

# Set T5 Tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained(t5_size, return_dict = True)

Train Batch Size: 4
Generate Batch Size: 30
Learning Rate: 0.0001
T5 Model Type: t5-base
T5 Task Name: generate fake news: 


For Training and Generation we will use the Tensorflow Dataset 'ag_news_subset'. It contains a train set of 120K rows and a test set of 7600 rows.

Each row contains a 'title' which is a news paper headline and a 'description' which is a short part of the news paper article.

The 'title' will be used as input and the long text 'description' will be specified as output. This way we can train the model to generate 'fake news' based on a short input.

We will split the train set in to 2 equal parts each with 60K rows.

The first set will be used to train the T5 model on its task.
The second set will be used to generate new data (the fake news...) with input data that the T5 model has never seen before.

!! Note: I've experienced multiple times that on the initial download of the dataset an error occurs. If you run it again it will just work...

In [ ]:
# Define Splits
splits = tfds.even_splits('train', n = TRAIN_SPLITS)
print(f'Configured Splits: {splits}')

# Get data and datasets
ag_news_ds, info = tfds.load('ag_news_subset', split = splits, with_info = True, shuffle_files = False, as_supervised = False)
train_ds = ag_news_ds[0]
generate_ds = ag_news_ds[1]

# Dataset features
print(f'Show Dataset Features:\n {info.features}')

# Samples
total_train_samples = int(info.splits['train'].num_examples / TRAIN_SPLITS) 
total_generate_samples = int(info.splits['train'].num_examples / TRAIN_SPLITS) 

# Summary
print(f'Total Samples for Training: {total_train_samples}')
print(f'Total Samples for Generation: {total_generate_samples}')

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: ag_news_subset/1.0.0


Configured Splits: ['train[0%:50%]', 'train[50%:100%]']


INFO:absl:Load dataset info from /tmp/tmpqxpznxb3tfds
INFO:absl:Generating dataset ag_news_subset (/root/tensorflow_datasets/ag_news_subset/1.0.0)


INFO:absl:URL https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbUDNpeUdjb0wxRms already downloaded: reusing /root/tensorflow_datasets/downloads/ucexport_download_id_0Bz8a_Dbh9QhbUDNpeUdjb0wxj4g1umFAV8OV-uDwxSJR0LdxO_k1jxMuFWwAfNX9jos.


Shuffling and writing examples to /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteB2FLHV/ag_news_subset-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteB2FLHV/ag_news_subset-train.tfrecord. Shard lengths: [120000]


Shuffling and writing examples to /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteB2FLHV/ag_news_subset-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteB2FLHV/ag_news_subset-test.tfrecord. Shard lengths: [7600]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split ['train[0%:50%]', 'train[50%:100%]'], from /root/tensorflow_datasets/ag_news_subset/1.0.0


Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.
Show Dataset Features:
 FeaturesDict({
    'description': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=4),
    'title': Text(shape=(), dtype=tf.string),
})
Total Samples for Training: 60000
Total Samples for Generation: 60000


Next map the train_ds and generate_ds to use only the 'description' and 'title'. The train_ds is used for training the model. The generate_ds is used for generating the fake news.

In [ ]:
# Map and Decode Split(s)
def decode_example(example):
    decoded_example = info.features.decode_example(example)
    
    description = decoded_example['description']
    title = decoded_example['title']
    
    return title, description

# Map
train_ds = train_ds.map(decode_example, num_parallel_calls = tf.data.experimental.AUTOTUNE)
generate_ds = generate_ds.map(decode_example, num_parallel_calls = tf.data.experimental.AUTOTUNE)

Lets look at some tokenized samples from the train_ds and generate_ds

In [ ]:
# Train: Show Input and Output Samples encoded
train_ds_numpy = tfds.as_numpy(train_ds.take(2))

for sample in train_ds_numpy:
    # Get title and description as strings
    title = sample[0].decode('utf-8')
    description = sample[1].decode('utf-8')
    
    # Encode with special tokens and use maximum length
    input_encoded = t5_tokenizer.encode_plus(title, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
    output_encoded = t5_tokenizer.encode_plus(description, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
    
    # Print...
    print(f'Title: {title}')
    print(f'Input - Title Encoded: {input_encoded}')
    print(f'Description: {description}')
    print(f'Output - Description Encoded: {output_encoded}\n')

Title: AMD Debuts Dual-Core Opteron Processor
Input - Title Encoded: {'input_ids': [22806, 374, 2780, 7, 17338, 18, 13026, 15, 4495, 449, 106, 10272, 127, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Test: Show Input and Output Samples encoded
generate_ds_numpy = tfds.as_numpy(generate_ds.take(2))

for batch in generate_ds_numpy:
    # Get title and description as strings
    title = batch[0].decode('utf-8')
    description = batch[1].decode('utf-8')
    
    # Encode with special tokens and use maximum length
    input_encoded = t5_tokenizer.encode_plus(title, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
    output_encoded = t5_tokenizer.encode_plus(description, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
            
    # Print...
    print(f'Title: {title}')
    print(f'Input - Title Encoded: {input_encoded}')
    print(f'Description: {description}')
    print(f'Output - Description Encoded: {output_encoded}\n')

Title: XM Strikes Back
Input - Title Encoded: {'input_ids': [3, 4, 329, 5500, 5208, 7, 3195, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Perform processing of the train_ds and prepare the data for model training.

In [ ]:
# Placeholders input
input_ids = np.zeros((total_train_samples, MAX_LEN), dtype='int32')
input_masks = np.zeros((total_train_samples, MAX_LEN), dtype='int32')

# Placeholders output
output_ids = np.zeros((total_train_samples, MAX_LEN), dtype='int32')
output_masks = np.zeros((total_train_samples, MAX_LEN), dtype='int32')

# Process Tensorflow Dataset as Numpy ... otherwise not possible to process tokenization.
train_ds_numpy = tfds.as_numpy(train_ds)

for index, sample in tqdm(zip(range(total_train_samples), train_ds_numpy), total = total_train_samples):
    
    # Get title and description as strings
    title = sample[0].decode('utf-8')
    description = sample[1].decode('utf-8')
    
    # Process Input
    input_encoded = t5_tokenizer.encode_plus(task_name + title, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
    input_ids_sample = input_encoded['input_ids']
    input_ids[index,:] = input_ids_sample
    attention_mask_sample = input_encoded['attention_mask']
    input_masks[index,:] = attention_mask_sample

    # Process Output
    output_encoded = t5_tokenizer.encode_plus(description, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
    output_ids_sample = output_encoded['input_ids']
    output_ids[index,:] = output_ids_sample
    attention_mask_sample = output_encoded['attention_mask']
    output_masks[index,:] = attention_mask_sample

Create the Keras Model to be used for T5.

In [ ]:
class KerasTFT5ForConditionalGeneration(TFT5ForConditionalGeneration):
    def __init__(self, *args, log_dir=None, cache_dir= None, **kwargs):
        super().__init__(*args, **kwargs)

        self.loss_tracker= tf.keras.metrics.Mean(name='loss') 
    
    @tf.function
    def train_step(self, data):
        x = data[0]
        y = x['labels']
        y = tf.reshape(y, [-1, 1])
        with tf.GradientTape() as tape:
            outputs = self(x, training=True)
            loss = outputs[0]
            logits = outputs[1]
            loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, self.trainable_variables)
            
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.loss_tracker.update_state(loss)        
        self.compiled_metrics.update_state(y, logits)
        metrics = {m.name: m.result() for m in self.metrics}
        
        return metrics

    def test_step(self, data):
        x = data[0]
        y = x["labels"]
        y = tf.reshape(y, [-1, 1])
        output = self(x, training=False)
        loss = output[0]
        loss = tf.reduce_mean(loss)
        logits = output[1]
        
        self.loss_tracker.update_state(loss)
        self.compiled_metrics.update_state(y, logits)
        
        return {m.name: m.result() for m in self.metrics}

Create a callback to save the model weights to storage.

In [ ]:
class SaveModel(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        print("\nSave Model Weights")

        # Save the entire model as a SavedModel.
        self.model.save_weights(WORK_DIR + 't5_base_model.h5')

Finally create and compile the model. Show the summary. Set the final input_data and perform the training.

In [ ]:
# Perform training only if specified
if PERFORM_TRAINING:
        
    # Create Model
    with strategy.scope():
        model = KerasTFT5ForConditionalGeneration.from_pretrained(t5_size, config = t5_config)
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LR), 
                      metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name = 'accuracy')])

    # Summary
    model.summary()

    # Set Input
    input_data = {'input_ids': input_ids, 'labels': output_ids, 'attention_mask': input_masks, 'decoder_attention_mask': output_masks}

    # Fit Model
    model.fit(input_data,
              epochs = EPOCHS, 
              batch_size = BATCH_SIZE, 
              verbose = VERBOSE,
              shuffle = True,
              callbacks = [SaveModel()],
              use_multiprocessing = False,
              workers = 4)

All model checkpoint layers were used when initializing KerasTFT5ForConditionalGeneration.

All the layers of KerasTFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use KerasTFT5ForConditionalGeneration for predictions without further training.


Model: "keras_tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275008 
Total params: 222,903,554
Trainable params: 222,903,552
Non-trainable params: 2
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f812a3632a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.auto

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).WARNING:tensorflow:AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f80fce806c8>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f812a3632a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when callin

15000/15000 [==============================] - 10381s 689ms/step - accuracy: 0.9636 - loss: 0.2784

Save Model Weights


If GENERATE_TEXT is true than create the model and load the weights file.

In [ ]:
if GENERATE_TEXT:        
    # Create Model
    with strategy.scope():
        model = KerasTFT5ForConditionalGeneration.from_pretrained(t5_size, config = t5_config)
        model.compile(optimizer = tf.keras.optimizers.Adam(), 
                      metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name = 'accuracy')])

    # Summary
    model.summary()

    # Load Weights
    model.load_weights(WORK_DIR + 't5_base_model.h5')

All model checkpoint layers were used when initializing KerasTFT5ForConditionalGeneration.

All the layers of KerasTFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use KerasTFT5ForConditionalGeneration for predictions without further training.


Model: "keras_tf_t5for_conditional_generation_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275008 
Total params: 222,903,554
Trainable params: 222,903,552
Non-trainable params: 2
_________________________________________________________________


Lets show a few examples of the generated fake news. The 'title' is shown as is the original 'description'. The 'Generated Fake News' is the output generated by the T5 model.

In [ ]:
if GENERATE_TEXT:
    # Test: Show Input and Output Samples encoded
    generate_samples = 4
    generate_ds_numpy = tfds.as_numpy(generate_ds.take(generate_samples))

    for index, sample in zip(range(generate_samples), generate_ds_numpy):
        # Get title and description as strings
        title = sample[0].decode('utf-8')
        description = sample[1].decode('utf-8')

        print(f'\n\n========= Sample:  {index}')
        print(f'Title: {title}')
        print(f'Description: {description}')

        # Encode with Special Tokens
        input_encoded = t5_tokenizer.encode_plus(task_name + title, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length', return_tensors = 'tf')
        
        # Generate FakeNews
        generated_fakenews = model.generate(input_encoded['input_ids'], 
                                          attention_mask = input_encoded['attention_mask'], 
                                          max_length = MAX_LEN, 
                                          top_p = 0.96, 
                                          top_k = 256, 
                                          temperature = 1.3,
                                          num_beams = 2, 
                                          num_return_sequences = 1, 
                                          repetition_penalty = 1.3,
                                          length_penalty = 1.3)

        for mapping in generated_fakenews.numpy():
            decoded_mapping = t5_tokenizer.decode(mapping, skip_special_tokens = True)
            print(f"    Generated Fake News: {decoded_mapping}")



========= Sample:  0
Title: XM Strikes Back
Description: Two weeks after its rival Sirius Satellite Radio (Nasdaq: SIRI) grabbed headlines by signing Howard Stern, XM Satellite Radio (Nasdaq: XMSR) disclosed late yesterday that it 
    Generated Fake News: XM Communications Inc. (XM.N: Quote, Profile, Research) on Tuesday said it will be back in business for the first time in three years after announcing that it had acquired a rival company.


========= Sample:  1
Title: RIGHT GAINS IN LITHUANIA ELECTIONS
Description: Right-wing parties have scored a surprise success in Lithuania #39;s parliamentary elections on the weekend. The right-wing Conservative Party and Liberal Center Union party won 43 seats together in the 141-member parliament.
    Generated Fake News: SAN FRANCISCO (Reuters) - The United States is poised for a second straight victory over the United States in the presidential election on Tuesday, but it's not clear how much of a difference the results will have to make.


Use the generate_ds to prepare the final dataframe that will be saved to disk after the fake news generation.

In [ ]:
if GENERATE_TEXT:
    # Placeholders
    titles, descriptions = [], []
    
    # Process Tensorflow Dataset as Numpy ... otherwise not possible to process tokenization.
    generate_ds_numpy = tfds.as_numpy(generate_ds)

    for index, sample in tqdm(zip(range(total_generate_samples), generate_ds_numpy), total = total_generate_samples):
        # Get title and description as strings
        titles.append(sample[0].decode('utf-8'))             # title
        descriptions.append(sample[1].decode('utf-8'))       # description = 

    # Create Dataframe
    df = pd.DataFrame()
    df['title'] = titles
    df['description'] = descriptions
    df['generated'] = ''

    # Summary
    print(df.head())
    print(df.shape)


                                               title  ... generated
0                                    XM Strikes Back  ...          
1                 RIGHT GAINS IN LITHUANIA ELECTIONS  ...          
2  Spain sweeps opening singles from US in Davis ...  ...          
3         Before the Bell: GE, Sirius Slip (Reuters)  ...          
4                  Microsoft takes on desktop search  ...          

[5 rows x 3 columns]
(60000, 3)


Perform the text generation based on the prepared dataframe. Note that the 'title' is used as input. The generated fake news will be stored in the dataframe 'generated' column.

The dataframe is saved to storage to be used for further use in the follow up notebooks.

In [ ]:
if GENERATE_TEXT:
    text_list = None
    generated = []

    for index, row in tqdm(zip(range(total_generate_samples), df.iterrows()), total = total_generate_samples):
        index += 1

        if text_list is None:
            text_list = []

        # Prep input text
        text_list.append(task_name + row[1]['title'])
        
        if index % GENERATE_BATCH_SIZE == 0:
            # Batch Encode with Special Tokens
            textlist_encoded = t5_tokenizer.batch_encode_plus(text_list, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length', return_tensors = 'tf')
            
            input_ids = textlist_encoded['input_ids']
            
            # Generate FakeNews
            generated_fakenews = model.generate(input_ids, 
                                                max_length = MAX_LEN, 
                                                top_p = 0.96, 
                                                top_k = 256, 
                                                temperature = 1.3,
                                                num_beams = 1, 
                                                num_return_sequences = 1, 
                                                repetition_penalty = 1.3)
            
            for mapping in generated_fakenews.numpy():
                generated.append(t5_tokenizer.decode(mapping, skip_special_tokens = True))

            # Reset Text List
            text_list = []

    # Generate Final File
    df['generated'] = generated
    df.to_csv(WORK_DIR + 't5_generated_fake_news.csv')
    print(df.head())


                                               title  ...                                          generated
0                                    XM Strikes Back  ...  XM Communications Inc., the world #39;s larges...
1                 RIGHT GAINS IN LITHUANIA ELECTIONS  ...  ATHENS (Reuters) - The United States is in the...
2  Spain sweeps opening singles from US in Davis ...  ...  Spain beat the US 6-4, 6-2, 6-2 in the Davis C...
3         Before the Bell: GE, Sirius Slip (Reuters)  ...  Reuters - The price ofsilver on the Internet i...
4                  Microsoft takes on desktop search  ...  Microsoft Corp. has taken a more aggressive ap...

[5 rows x 3 columns]
